1) Import Packages

In [21]:
import numpy as np
import scipy as sp
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
import category_encoders as ce
import xgboost as xgb
import matplotlib.pyplot as mpl
from random import randint, choice

2) Load data, and simplify/clean it. This includes: only selecting variables needed; encoding the species; dropping NaN data in both censuses; converting to numpy arrays; altering such that labels represent growth.

In [22]:
t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv")
t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")

# COMMENT OUT IF USING ALL OF THE DATA
t1 = t1.loc[t1['quadrat']<=50]
t2 = t2.loc[t2['quadrat']<=50]

# simplify columns and column names
expected_labels = t2[['treeID', 'dbh']]
expected_labels = expected_labels.rename(columns={"dbh": "dbh2", "treeID": "treeID2"})

featurest1 = t1[['treeID', 'sp', 'dbh', 'gx', 'gy']]
featurest1 = featurest1.rename(columns={"dbh": "dbh1", "treeID":"treeID1"})

# encode the species into binary features and drop rows with any NaN values
encoder= ce.BinaryEncoder(cols=['sp'],return_df=True)
featurest1 = encoder.fit_transform(featurest1)

df_combined = pd.concat([featurest1, expected_labels], axis=1)
df_combined_clean = df_combined.dropna()

# split back up and convert the dataframes to numpy arrays
df_clean = df_combined_clean[featurest1.columns]
labels_clean = df_combined_clean[expected_labels.columns]

featurest1 = df_clean.to_numpy()
expected_labels = labels_clean.to_numpy()

featurest1 = featurest1.astype(np.float32)
expected_labels = expected_labels.astype(np.float32)


# change from numpy using round and then multiply by 5 (np a round)
#featurest1[:,10] = np.around(featurest1[:,10]/5)* 5

/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_11158/896708291.py:1: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv")
/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_11158/896708291.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")


3) Edit variable "a" to represent how many columns the species encoding uses. This will impact calculations further in the file.

In [23]:
# HOW MANY COLS TO ENCODE SPECIES?
a = 8

In [24]:
# ensure that there is only positive growth, represented by CHANGE (not total dbh)
expected_labels[:,1] = expected_labels[:,1] - featurest1[:,a+1]
expected_labels = np.where (expected_labels<0, 0, expected_labels)

***INTRINSIC MODEL BELOW***

In [44]:
X_train_in, X_test_in, y_train_in, y_test_in = train_test_split(featurest1, expected_labels, test_size=0.3)

train_feats_in = X_train_in[:,1:a+2]
train_labels_in = y_train_in[:,1]

test_labels_in = y_test_in[:,1]
test_feats_in = X_test_in[:,1:a+2]

In [45]:
rf_in = RandomForestRegressor()
rf_in.fit(train_feats_in[:,1:], train_labels_in)
preds_in = rf_in.predict(test_feats_in[:,1:])
error_in = r2_score(test_labels_in, preds_in)
print (error_in)

0.05120801243011064


4) Separate out the x coordinates and y coordinates. Create a matrix with the distance from each focal tree to its nearest neighbors.

In [25]:
ids = featurest1[:,0]
# X WOULD BE a +1, y would be a+2
x_coordinates = featurest1[:, a+2]  
y_coordinates = featurest1[:, a+3]
coord_matrix = np.column_stack((x_coordinates, y_coordinates))
spatial_tree = sp.spatial.KDTree(coord_matrix)

5) Create "blanks," or accumulators for the nearest neighbors to accumulate themselves in. Though we are looking at the 5 nearest neighbors, include 6 columns because the first closest neighbor will be itself. We will later remove this.

In [26]:
nn_dist_matrix2 = np.zeros((len(coord_matrix),6))
nn_ind_matrix2 = np.zeros((len(coord_matrix),6))
nn_feats = featurest1[:, 0:a+2]
#nn_feats = np.column_stack((featurest1[:, 0:a+1], featurest1[:,a+3]))
#species, dbh
feats_matrix = np.zeros((len(coord_matrix),(6*(a+2))))

6) Loop through the coordinate matrix, and add the features of the neighbors to a large data structure. In this data structure, each row represents ONE tree. The first  features of this row are the focal tree's characteristics. The features further to the right are the features of the neighbors. 

In [28]:
for i, tree in enumerate(coord_matrix):
    dist2, ind2 = spatial_tree.query(tree, k=6)
    nn_ind_matrix2[i] = ids[ind2]
    nn_dist_matrix2[i]= dist2

    nn_row = nn_feats[i].reshape(1,a+2)
    inc = 0
    for j in nn_ind_matrix2[i][1:]:
        row_ind = np.where(featurest1[:,0] == j)
        real_row = (featurest1[row_ind])
        distance = dist2[1:][inc].reshape(1,1)
        dbh = real_row[:,a+1].reshape(1,1)
        nn_row = np.hstack((nn_row, distance, real_row[:,1:a+1],dbh))
        inc += 1
    
    feats_matrix[i] = nn_row
    
#nn_ind_matrix2 = nn_ind_matrix2[:,1:]
#nn_dist_matrix2 = nn_dist_matrix2[:,1:]
# remove the columns with the focal tree's OWN distance and IDs

7) Split into test and train data using sklearn.

In [29]:
X_train, X_test, y_train, y_test = train_test_split(feats_matrix, expected_labels, test_size=0.3)

train_feats = X_train[:,1:]
train_labels = y_train[:,1]

test_labels = y_test[:,1]
test_feats = X_test[:,1:]

8) Do a hyperparameter search for ideal hyperparameters for random forest model.

In [30]:
# HYPERPARAMETER SEARCH
n_estimators = np.arange(100,200,400)
max_depth = [7,8,9]
random_grid = {'max_depth': max_depth, 'n_estimators': n_estimators}
rf = RandomForestRegressor()
rf_grid = GridSearchCV(rf, param_grid= random_grid, scoring='r2', cv=5 )
rf_grid.fit(test_feats[:,1:], test_labels)
depth = rf_grid.best_params_['max_depth']
est = rf_grid.best_params_['n_estimators']

9) Complete training and testing using these hyperparameters.

In [31]:
rf = RandomForestRegressor(max_depth=depth, n_estimators=est)
rf.fit(train_feats[:,1:], train_labels)
preds = rf.predict(test_feats[:,1:])
error1a = r2_score(test_labels, preds)
print (error1a)

0.0600466722529327


10) **Begin to introduce randomization**: Create a matrix where each row is a different permutation of the series [1,2,3,4,5]. Here, we do this 20 times (ie: 20 rows, 20 randomizations).

In [32]:
# generate 20 random sequences of 1 to 5 - COMPLETE
rng = np.random.default_rng()
matrix_order = np.zeros((20,5))

shuffle_this = np.array([0, 1, 2, 3, 4])
for i in range(20):
  current_row = rng.permutation(shuffle_this)
  matrix_order[i] = current_row

11) Again, create sort of "blanks," or accumulators for the loop to use later on. These will eventually become our larger, randomized data structure.

In [33]:
nn_dist_matrix3 = np.zeros((len(coord_matrix),6))
nn_ind_matrix3 = np.zeros((len(coord_matrix),6))
nn_feats2 = featurest1[:, 0:a+2]

12) Iterate through those data structures, and create the correct, randomized structure. This way, the model will learn to use the DISTANCE associated with each tree, not just its index. (Sort of creating synthetic data)

In [34]:
# Start with feats_matrix above. Then create similar matrices for each of the above
feats_matrix_big = feats_matrix.copy()

# of columns should be a*6

zer_indices = range(a+2,2*a+4)
one_indices = range (2*a+4,3*a+6)
two_indices = range (3*a+6,4*a+8)
three_indices = range (4*a+8,5*a+10)
four_indices = range (5*a+10,6*a+12)

# the long number passing into range is ust n=20 here, but this would make it less hard-coded in
for i in range(np.shape(matrix_order)[0]):
    # loop through each permutation.
    feats_matrix_small = feats_matrix.copy()
    perm_indices = [range(a+2)]
    current_perm = matrix_order[i]
    for x in current_perm:
        if x == 0:
          perm_indices.append(zer_indices)
        elif x == 1:
           perm_indices.append(one_indices)
        elif x == 2:
           perm_indices.append(two_indices)
        elif x == 3:
           perm_indices.append(three_indices)
        elif x == 4:
           perm_indices.append(four_indices)
    flattened = []
    for sublist in perm_indices:
       for item in sublist:
          flattened.append(item)
    feats_matrix_small = feats_matrix_small[:,flattened]
    feats_matrix_big = np.vstack((feats_matrix_big,feats_matrix_small))

13) Create a data structure to store the corresponding y labels. Essentially, just vertically append them to each other, the same number of times as permutations we have above.

In [35]:
#  - THIS WORKS! # of rows = 138x20
expected_labels2 = expected_labels.copy()
for i in range(20):
    expected_labels2 = np.vstack((expected_labels2, expected_labels))

14) Train a new random forest regressor on this? Or use old one?

In [36]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(feats_matrix_big, expected_labels2, test_size=0.3)

train_feats2 = X_train2[:,1:]
train_labels2 = y_train2[:,1]

test_labels2 = y_test2[:,1]
test_feats2 = X_test2[:,1:]

In [37]:
# HYPERPARAMETER SEARCH
n_estimators = np.arange(100,200,400)
max_depth = [7,8,9]
random_grid = {'max_depth': max_depth, 'n_estimators': n_estimators}
rf_big = RandomForestRegressor()
rf_grid_big = GridSearchCV(rf, param_grid= random_grid, scoring='r2', cv=5 )
rf_grid_big.fit(test_feats2[:,1:], test_labels2)
depth2 = rf_grid_big.best_params_['max_depth']
est2 = rf_grid_big.best_params_['n_estimators']

In [38]:
rf = RandomForestRegressor(max_depth=depth2, n_estimators=est2)
rf.fit(train_feats2[:,1:], train_labels2)
preds2 = rf.predict(test_feats2[:,1:])
error2 = r2_score(test_labels2, preds2)
print (error2)

0.6353821765173253


15) Use framework from diagnostics file to create a sorted list of species by decreasing abundance. This will allow us to call on certain species for testing, plotting, and printing. This is something that could be made into its own file in the future.

In [40]:
names = encoder.inverse_transform(featurest1)
sp_list = names.sp.unique()
sp_freq = names.sp.value_counts(sort=True).index.tolist()
sp_num = sp_list.shape[0]
#sp_mat = np.hstack((np.atleast_2d(sp_list).transpose(), (np.zeros((sp_num, a)))))

16) Create a data structure where each species tree has a DBH in the range 10-500, every 5 mm. This is "one_big_array."

In [42]:
all_dbhs = np.arange(10,501,5).transpose()
all_dbhs_big = np.arange(10,501,5).transpose()
list_of_all_sp = []

for i in range(sp_num):
    # 1) look at initial dataset and find the tree with that ID, then find its encoding
    # find tree ID of one instance of a species
    tree_id = names[names.sp == sp_freq[i]].treeID1
    one_tree_id = tree_id.iloc[0]
    encoding = feats_matrix_big[feats_matrix_big[:,0]==one_tree_id][0,1:a+1]
    sp_label = np.atleast_2d(encoding)
    sp_repeated = np.repeat(sp_label,99,axis=0)
    sp_i = np.column_stack((sp_repeated,all_dbhs))
    all_dbhs_big = np.vstack((all_dbhs_big, all_dbhs))
    list_of_all_sp.append(sp_i)

one_big_array = np.vstack(list_of_all_sp)
all_dbhs_big = (all_dbhs_big.flatten())[:a*3234] # can be represented by a*3234?

list_of_all_preds = []
for sp in list_of_all_sp:
    # adjust as if it had all zero neighbors
    sp_adjusted = np.column_stack((sp,np.zeros((99,5*(a+2)))))
    preds_diagnostic = rf_in.predict(sp_adjusted)
    list_of_all_preds.append(preds_diagnostic)